---
title: "线性方程组"
---

## 学习目标

* **问题**：给定 $A\in\mathbb{R}^{n\times n}$、$b\in\mathbb{R}^n$；求解 $Ax=b$。
* **直接法**（一次分解，多次回代）

  1. **高斯消元 ↔ LU 分解**；
  2. **主元选取（部分/全主元）**提升稳定性；
  3. **批量右端**：分解一次，复用解多个 $b$。
* **误差与稳定性**：条件数 $\kappa(A)$、残差 $\|r\|=\|b-A\hat x\|$、**后向稳定性**；
  $\displaystyle \frac{\|\hat x-x^*\|}{\|x^*\|}\lesssim \kappa(A)\cdot \frac{\|r\|}{\|b\|}$（启发式）。
* **迭代法**：Jacobi、Gauss–Seidel 的**收敛条件**（$\rho(T)<1$、对角占优等）；**共轭梯度（CG）**用于 SPD；
  **预条件**：Jacobi（对角）、IC/SSOR（概念） → 加速收敛。
* **工程**：**vmap/批量 RHS**。


# 一、直接法

## 高斯消元

取
$$
A=\begin{bmatrix}
2 & 1 & 1\\
4 & -6 & 0\\
-2 & 7 & 2
\end{bmatrix},\qquad \det A=-16\neq 0.
$$

**Step 1（消去第1列下方）**：主元 $a_{11}=2$。乘子
$$
\ell_{21}=\frac{4}{2}=2,\quad \ell_{31}=\frac{-2}{2}=-1.
$$

行变换
$$
R_2\leftarrow R_2-\ell_{21}R_1,\quad R_3\leftarrow R_3-\ell_{31}R_1
$$

得
$$
A^{(1)}=\begin{bmatrix}
2 & 1 & 1\\
0 & -8 & -2\\
0 & 8 & 3
\end{bmatrix}.
$$

**Step 2（消去第2列下方）**：主元 $a^{(1)}_{22}=-8$。乘子
$$
\ell_{32}=\frac{8}{-8}=-1,\quad R_3\leftarrow R_3-\ell_{32}R_2.
$$

得到上三角
$$
U=\begin{bmatrix}
2 & 1 & 1\\
0 & -8 & -2\\
0 & 0 & 1
\end{bmatrix}.
$$

把乘子收集到严格下三角并在对角补 1，得
$$
L=\begin{bmatrix}
1 & 0 & 0\\
\ell_{21} & 1 & 0\\
\ell_{31} & \ell_{32} & 1
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0\\
2 & 1 & 0\\
-1 & -1 & 1
\end{bmatrix}.
$$
可直接验证 $A=LU$。

::: callout-note
**求解 $Ax=b$**：先解 $Ly=Pb$（本例 $P=I$），再解 $Ux=y$。
:::

## 从消元到 LU

* 在不换行的理想情形：存在下三角 $L$（对角为 1）与上三角 $U$，使 $A=LU$。
* 通过消元系数 $l_{ik}=\displaystyle\frac{a_{ik}}{u_{kk}}$ 记录到 $L$，更新上三角为 $U$。

**主元选取**

* **部分主元**（列内最大绝对值交换到对角）：稳定常用，实现为 $PA=LU$。
* **全主元**（行列同时交换）：更稳但代价高，通常不必。


## 部分主元 LU

In [ ]:
import numpy as np

def lu_factor_pp(A):
    """
    LU with partial pivoting: PA = LU
    返回 L（含 1 的对角）、U、piv（行交换记录）
    """
    A = np.array(A, dtype=np.float64, copy=True)
    n = A.shape[0]
    piv = np.arange(n)
    for k in range(n-1):
        # 选列主元
        i = k + np.argmax(np.abs(A[k:, k]))
        if A[i, k] == 0:
            raise np.linalg.LinAlgError("Singular matrix")
        if i != k:
            A[[k, i], :] = A[[i, k], :]
            piv[[k, i]] = piv[[i, k]]
        # 消元
        A[k+1:, k] /= A[k, k]
        A[k+1:, k+1:] -= np.outer(A[k+1:, k], A[k, k+1:])
    # 拆出 L, U
    L = np.tril(A, -1) + np.eye(n)
    U = np.triu(A)
    return L, U, piv

def lu_solve(L, U, piv, b):
    """
    解 PAx = b
    b 可为 (n,) 或 (n,m)（批量右端）
    """
    b = np.array(b, dtype=np.float64, copy=False)
    n = U.shape[0]
    # 施加置换 P
    if b.ndim == 1:
        y = b[piv].copy()
        # 前代：Ly = Pb
        for i in range(n):
            y[i] -= L[i, :i] @ y[:i]
        # 回代：Ux = y
        x = y.copy()
        for i in range(n-1, -1, -1):
            x[i] = (x[i] - U[i, i+1:] @ x[i+1:]) / U[i, i]
        return x
    else:
        Y = b[piv, :].copy()
        for i in range(n):
            Y[i, :] -= L[i, :i] @ Y[:i, :]
        X = Y.copy()
        for i in range(n-1, -1, -1):
            X[i, :] = (X[i, :] - U[i, i+1:] @ X[i+1:, :]) / U[i, i]
        return X

**验证与对比**

In [ ]:
# 随机可逆矩阵与多 RHS
rng = np.random.default_rng(0)
A = rng.standard_normal((5, 5))
A = A + 5 * np.eye(5)  # 稳一点
B = rng.standard_normal((5, 3))
L, U, piv = lu_factor_pp(A)
X1 = lu_solve(L, U, piv, B)
X2 = np.linalg.solve(A, B)
print("||A X1 - B|| =", np.linalg.norm(A @ X1 - B))
print("||X1 - X2||  =", np.linalg.norm(X1 - X2))

## 主元：一个反例

In [ ]:
eps = 1e-12
A_bad = np.array([[eps, 1.0],[1.0, 1.0]], dtype=np.float64)
b     = np.array([1.0, 2.0])
# 无主元（手动实现时会爆精度） vs NumPy（内部含主元）
x_np = np.linalg.solve(A_bad, b)
print("cond(A_bad)≈", np.linalg.cond(A_bad), "  x_np =", x_np)

对于方程组
$$
\begin{cases} \epsilon x_1 + x_2 = 1 \\ x_1 + x_2 = 2 \end{cases}
$$

作差得 $(1-\epsilon)x_1 = 1$，所以 $x_1 = \frac{1}{1-\epsilon}$。

代入第二个方程得 $x_2 = 2 - x_1 = 2 - \frac{1}{1-\epsilon} = \frac{2(1-\epsilon)-1}{1-\epsilon} = \frac{1-2\epsilon}{1-\epsilon}$。

因为 $\epsilon = 10^{-12}$ 极小，所以 $x_1 \approx \frac{1}{1} = 1$，$x_2 \approx \frac{1}{1} = 1$。
NumPy 计算出的结果 $x_{\text{np}}$ 应该非常接近 $[1.0, 1.0]$。

::: callout-note
不选主元可能导致“增长因子”极大，舍入误差被成倍放大。**部分主元**是工程默认。NumPy 的 `np.linalg.solve` 函数在底层调用了高度优化的线性代数库（如 LAPACK），这些库在实现高斯消元（或 LU 分解）时，默认使用了部分主元选择（Partial Pivoting）等技术。
:::

# 二、条件数、残差与稳定性

## 条件数与残差不等式

在数值线性代数中，对于求解线性方程组 $Ax = b$ 的一个近似解 $\hat{x}$，我们通常关心它的误差 $e = \hat{x} - x^*$ 有多大。其中真解 $x^*$ 满足 $Ax^*=b$。然而，我们通常无法直接计算误差 $e$，因为我们不知道精确解。定义**残差**
$$
r=b-A\hat x.
$$

残差是衡量近似解 $\hat{x}$ 代入原方程后，等式不平衡的程度。

有
$$
A(\hat x-x^*)=r\ \Rightarrow\ \hat x-x^* = A^{-1}r.
$$

于是
$$
\frac{\|\hat x-x^*\|}{\|x^*\|}
\le \frac{\|A^{-1}\|\,\|r\|}{\|x^*\|}.
$$
又因为 $\|b\|=\|Ax^*\|\le \|A\|\,\|x^*\|\Rightarrow \|x^*\|\ge \frac{\|b\|}{\|A\|}$，

代回得到
$$
\boxed{\quad
\frac{\|e\|}{\|x^*\|}
\ \le\ \|A\|\,\|A^{-1}\| \cdot \frac{\|r\|}{\|b\|}
\ =\ \kappa(A)\cdot \frac{\|r\|}{\|b\|}.
\quad}
$$

如果矩阵 $A$ 是良态的（well-conditioned，即 $\kappa(A)$ 较小），那么不等式很有用。它表明相对误差（$\|e\|/\|x^*\|$）和相对残差（$\|r\|/\|b\|$）是同数量级的。残差可以作为误差的良好**启发式指标**。

如果矩阵 $A$ 是病态的（ill-conditioned，即 $\kappa(A)$ 很大），那么不等式虽然在数学上是严格正确的，但它失去了实用价值：这个上界对实际的误差大小的指导性不强。我们知道误差不会超过这个界限，但它可能在界限内的任何地方，使得残差和误差之间的关系非常模糊。

于是记为
$$
\frac{\|e\|}{\|x^*\|}\lesssim \kappa(A)\cdot \frac{\|r\|}{\|b\|}
$$

::: callout-important
残差小不一定意味着误差小。
:::

## 数值稳定性：**前向** vs **后向**

- **前向稳定（forward stable）**：算法输出 $\hat y$ 与真值 $y=f(x)$ 的**差**很小：
  $$
  \frac{\|\hat y-y\|}{\|y\|}=O(u),
  $$

  其中 $u$ 是机器精度。**注意**：对**病态问题**（$\kappa$ 大），任何算法的前向误差都可能被放大。

- **后向稳定（backward stable）**：算法输出 $\hat y$ 可视为对**略微扰动**的输入 $\tilde x=x+\delta x$ 的精确值：
  $$
  \hat y = f(\tilde x),\qquad \frac{\|\delta x\|}{\|x\|}=O(u).
  $$

  这意味着算法“**只把错误推回输入**”，问题若本身病态，前向误差仍会大，但算法本身是可信的。

- **典型例子**  
  - **高斯消元 + 部分主元（GEPP）**：对绝大多数矩阵是**后向稳定**的；可以解释为在 $(A+\delta A)\hat x=b$ 上**精确**，且 $\|\delta A\|/\|A\|=O(u)$（常数依赖增长因子）。  
  - **Horner 法**（多项式求值）是后向稳定的；  
  - **天真相加**（同号大数与小数相加）可能**前向不稳定**（严重抵消）。

## 实验：小残差≠解的误差小

In [ ]:
import numpy as np

# 构造病态 SPD（调节 kappa 控制条件数）
def make_spd_with_cond(n, kappa, seed=0):
    rng = np.random.default_rng(seed)
    Q, _ = np.linalg.qr(rng.standard_normal((n,n)))
    vals = np.logspace(0, np.log10(kappa), n)   # [1, kappa]
    A = (Q * vals) @ Q.T
    return (A + A.T)/2

n = 40
kappa = 1e10
A = make_spd_with_cond(n, kappa)
x_true = np.ones(n)
b = A @ x_true

# 用稳定的库解
x_hat = np.linalg.solve(A, b)

rel_res = np.linalg.norm(b - A@x_hat)/np.linalg.norm(b)
rel_err = np.linalg.norm(x_hat - x_true)/np.linalg.norm(x_true)
print(f"κ(A)≈{np.linalg.cond(A):.2e}")
print(f"relative residual   ≈ {rel_res:.2e}")
print(f"relative solution error ≈ {rel_err:.2e}")
print(f"κ(A) * rel_res      ≈ {np.linalg.cond(A)*rel_res:.2e}  (与上式同量级)")

尽管**残差**达到 1e-16 量级，**解误差**仍可能在 1e-6 甚至更大；这正是 $\kappa(A)$ 的放大效应。

> 这不是算法“坏”，而是问题**病态**：**后向稳定**保证我们求解的是“稍微扰动过的 $A$”的精确解。

::: callout-tip

* 高斯消元的**乘子**就是 $L$ 的下三角元素；最终 $A=LU$（或 $PA=LU$）。
* **部分主元**是抗增长因子、抗舍入误差。
* 条件数给出残差到解误差的放大界：$\text{forward} \lesssim \kappa \times \text{residual}$。
* **后向稳定** ≠ **前向误差小**；它意味着“算法把错误推回输入”，问题若病态，前向误差依然大。
:::

## 比较残差与解误差

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.sans-serif"] = ["PingFang SC", "Arial Unicode MS"]
plt.rcParams["axes.unicode_minus"] = False


def make_spd_with_cond(n, kappa):
    # A = Q diag(λ) Q^T，λ 在 [1, kappa] 对数均匀
    rng = np.random.default_rng(0)
    M = rng.standard_normal((n, n))
    Q, _ = np.linalg.qr(M)
    vals = np.logspace(0, np.log10(kappa), n)
    A = (Q * vals) @ Q.T  # 等价 Q diag(vals) Q^T
    return (A + A.T) / 2


def experiment(n=60, kappas=(1e1, 1e3, 1e6)):
    rng = np.random.default_rng(1)
    errs = []
    ress = []
    ks = []
    for kappa in kappas:
        A = make_spd_with_cond(n, kappa)
        x_true = rng.standard_normal(n)
        b = A @ x_true
        x_hat = np.linalg.solve(A, b)
        res = np.linalg.norm(b - A @ x_hat) / np.linalg.norm(b)
        err = np.linalg.norm(x_hat - x_true) / np.linalg.norm(x_true)
        ks.append(kappa)
        ress.append(res)
        errs.append(err)
        print(f"kappa≈{kappa:.1e}  rel_res={res:.2e}  rel_err={err:.2e}")
    plt.figure(figsize=(5.4, 3.4))
    plt.loglog(ks, ress, "-o", label="relative residual")
    plt.loglog(ks, errs, "-o", label="relative solution error")
    plt.xlabel("target κ(A)")
    plt.grid(True, which="both", alpha=0.3)
    plt.legend()
    plt.title("残差与解误差 vs 条件数（SPD示例）")
    plt.tight_layout()
    plt.show()


experiment()

# 三、迭代法

## 谱半径与收敛
- **定义**：矩阵 $T\in\mathbb{R}^{n\times n}$ 的**谱半径**（spectral radius）
  $$
  \rho(T)=\max\{\,|\lambda|:\lambda\ \text{是}\ T\ \text{的特征值}\,\}.
  $$
- **基本事实**（Gelfand 公式）：
  $$
  \rho(T)=\lim_{k\to\infty}\|T^k\|^{1/k}\quad(\text{任一相容矩阵范数}).
  $$
- **收敛判据**（线性不动点迭代）  
  考虑线性迭代
  $$
  x_{k+1}=T\,x_k+c.
  $$
  若存在真解 $x^*$ 满足 $x^*=T x^*+c$，令误差 $e_k=x_k-x^*$，则
  $$
  e_{k+1}=T\,e_k=T^{k+1}e_0.
  $$
  因此
  $$
  \|e_k\|\le \|T\|^k\,\|e_0\|,\qquad \text{若 }\rho(T)<1\ \Rightarrow\ T^k\to 0.
  $$
  **定理**：对线性定系数迭代，$\rho(T)<1$**当且仅当**对任意初值 $x_0$，$x_k\to x^*$。

> 证明思路：  
> “$\Leftarrow$” 若有某初值不收敛，则存在特征向量方向不衰减；  
> “$\Rightarrow$” 用 Neumann 级数 $(I-T)^{-1}=\displaystyle\sum_{j\ge0}T^j$ 或 Gelfand 公式得 $T^k\to 0$。


## 一般迭代格式
把 $A$ 拆分为
$$
A=M-N,\qquad M\ \text{可逆}.
$$
把线性方程 $Ax=b$ 写作
$$
Mx=(N x)+b \ \Rightarrow\ x=M^{-1}Nx+M^{-1}b.
$$
得到**一般的（stationary）迭代**：
$$
\boxed{\quad x_{k+1}=M^{-1}N\,x_k+M^{-1}b.\quad}
$$

- **迭代矩阵**：定义 $T=M^{-1}N$。  
- **误差传播**：$e_{k+1}=T e_k$。  
- **残差传播**（记 $r_k=b-Ax_k$）：
  $$
  r_{k+1}=b-Ax_{k+1}=(I-AM^{-1})\,r_k.
  $$

- **收敛**：$\rho(T)<1$（必要且充分）。


> **把迭代看作“预条件 Richardson”**  
> 也常写作
> $$
> x_{k+1}=x_k+M^{-1}(b-Ax_k)=x_k+M^{-1}r_k,
> $$
> 视 $M$ 为一种**预条件子**（近似 $A$ 且易解）。


## 矩阵“拆分”

记 $A=D+L+U$，其中

- $D=\mathrm{diag}(A)$ 对角阵
- $L$ 严格下三角
- $U$ 严格上三角

均按**原符号**分解。

1) **Jacobi**（每步只用旧解的其余分量）
$$
M=D,\quad N=-(L+U)\quad\Rightarrow\quad
\boxed{~x_{k+1}=D^{-1}\bigl(b-(L+U)x_k\bigr)~}
$$

迭代矩阵
$$
\boxed{~T_J=-D^{-1}(L+U).~}
$$

2) **Gauss–Seidel（GS）**（用“最新”的下三角更新）
$$
M=D+L,\quad N=-U\quad\Rightarrow\quad
\boxed{~(D+L)\,x_{k+1}=b-Ux_k~}
$$

显式写成分量形式（按行更新）即为经典 GS。

> **收敛条件（常见充分条件）**  
>
> - 若 $A$ **严格对角占优**（按行或按列），则 Jacobi 与 GS 都收敛（$\rho(T)<1$）。  
> - 若 $A$ **对称正定（Symmetric Positive-Definite，SPD）**，则 **GS 收敛**；Jacobi 常也收敛但不保证最快。  
> - 一般地，GS **通常收敛更快**（常见理论界：在某些条件下 $\rho(T_{GS})\le \rho(T_J)^2$）。

3) **SOR（超松弛）**  
$(D+\omega L)x_{k+1}=\omega b+\bigl[(1-\omega)D-\omega U\bigr]x_k$，  
$0<\omega<2$ 时对 SPD 常收敛；$\omega$ 可优化。


## 误差界与停止准则

- **误差界**：若选定相容范数 $\|\cdot\|$ 使 $\|T\|\le q<1$，则
  $$
  \|e_k\|\le q^k\|e_0\|,\qquad
  \|x_k-x^*\|\le \frac{q}{1-q}\,\|x_k-x_{k-1}\|.
  $$
- **常用停止准则**：
  $$
  \frac{\|r_k\|}{\|b\|}\le \varepsilon,\quad\text{或}\quad
  \frac{\|x_k-x_{k-1}\|}{\|x_k\|}\le \varepsilon.
  $$
  残差与误差成正比的常数取决于 $\kappa(A)$：病态时残差小≠误差小。


## Jacobi / GS 的分量公式

**Jacobi**：

第 $i$ 行
$$
a_{ii}x^{k+1}_i=b_i-\sum_{j\ne i}a_{ij}\,x^k_j
\quad\Rightarrow\quad
\boxed{~x^{k+1}_i=\frac{1}{a_{ii}}\Bigl(b_i-\sum_{j\ne i}a_{ij}\,x^k_j\Bigr).~}
$$

**Gauss–Seidel**：
$$
a_{ii}x^{k+1}_i
=b_i-\sum_{j<i}a_{ij}\,x^{k+1}_j-\sum_{j>i}a_{ij}\,x^k_j,
$$
$$
\boxed{~x^{k+1}_i=\frac{1}{a_{ii}}
\Bigl(b_i-\sum_{j<i}a_{ij}\,x^{k+1}_j-\sum_{j>i}a_{ij}\,x^k_j\Bigr).~}
$$
即：从 $i=1$ 到 $n$ 顺序更新，使用“刚算出的新值”。


## 谱半径与收敛：算例

取
$$
A=\begin{bmatrix}
4 & 1\\
2 & 3
\end{bmatrix},\quad b=\begin{bmatrix}1\\1\end{bmatrix},\quad
D=\begin{bmatrix}4&0\\0&3\end{bmatrix},\
L=\begin{bmatrix}0&0\\2&0\end{bmatrix},\
U=\begin{bmatrix}0&1\\0&0\end{bmatrix}.
$$

- **Jacobi**：$T_J=-D^{-1}(L+U)=\begin{bmatrix}0&-1/4\\-2/3&0\end{bmatrix}$。  
  特征值 $\lambda=\pm\sqrt{\tfrac{1}{6}}$，$\rho(T_J)=\sqrt{1/6}\approx 0.408<1$ ⇒ 收敛。

- **GS**：$T_{GS}=-(D+L)^{-1}U$。  
  $(D+L)^{-1}=\begin{bmatrix}1/4&0\\-2/12&1/3\end{bmatrix}$，
  $T_{GS}=\begin{bmatrix}0&-1/4\\0&1/6\end{bmatrix}$，$\rho(T_{GS})=1/6\approx 0.167$ ⇒ **更快**。

结论：$\rho(T_{GS})<\rho(T_J)$ ⇒ GS 比 Jacobi 收敛快。


### 伪代码：**一般拆分迭代**与 Jacobi/GS 特例

**一般迭代：$x_{k+1}=M^{-1}(N x_k+b)$**
```
procedure StationaryIter(A, b, split=(M, N), x0, tol, itmax):
x ← x0
for k = 0..itmax-1:
x_new ← solve(M, N*x + b)     # 解 M x_new = N x + b
if norm(b - A*x_new)/norm(b) < tol:
return x_new, k+1
x ← x_new
return x, itmax

```

**Jacobi**（用 `solve(M,·)` 退化为“按对角元素相除”）
```

M ← diag(A);  N ← -(L+U)
x_{k+1} ← (b - (L+U) x_k) ./ diag(A)

```

**Gauss–Seidel**（前代形式）
```

M ← D + L;  N ← -U
解 (D+L) x_{k+1} = b - U x_k   # 逐行或三角前代

```

### 计算谱半径并验证收敛

In [ ]:
import numpy as np

def spectral_radius(T):
    lam = np.linalg.eigvals(T)
    return np.max(np.abs(lam))

def jacobi_matrix(A):
    A = np.asarray(A, float)
    D = np.diag(np.diag(A))
    R = A - D
    return -np.linalg.inv(D) @ R

def gs_matrix(A):
    A = np.asarray(A, float)
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A,  1)
    return -np.linalg.inv(D+L) @ U

A = np.array([[4.,1.],[2.,3.]])
TJ = jacobi_matrix(A)
TG = gs_matrix(A)
print("rho(T_J) =", spectral_radius(TJ))
print("rho(T_GS) =", spectral_radius(TG))

::: callout-note
若 $\rho(T)<1$ 则对任意 $x_0$ 收敛；可用上述 `T` 验证不同拆分的优劣。
:::

## 与 CG/PCG 的关系
- Jacobi/GS/SOR **是定系数的“一般迭代”**；  
- **CG/PCG** 是**非定系数（Krylov）迭代**，只适用于 **SPD**，但收敛更快、可用预条件（见本章后续与附录）。


## 小结

* **先看谱半径**：$\rho(T)<1$ 是线性迭代收敛的标准。
* **一般格式**来自 $A=M-N$，误差递推 $e_{k+1}=M^{-1}N\,e_k$。
* **Jacobi、GS** 是两种典型拆分，严格对角占优/SPD 情况下有收敛保证。
* **收敛速度**：$|e_k|\approx O(\rho(T)^k)$，$\rho$ 越小越快，GS 往往优于 Jacobi。
* **停止准则**：残差/步长监控并结合 $\kappa(A)$ 解读。


## 完整迭代实现

In [ ]:
def jacobi(A, b, x0=None, tol=1e-10, itmax=10_000):
    A = np.asarray(A, float); b = np.asarray(b, float)
    n = b.size; x = np.zeros(n) if x0 is None else x0.copy()
    D = np.diag(A); R = A - np.diagflat(D)
    hist = []
    for k in range(itmax):
        x = (b - R @ x) / D
        r = b - A @ x
        hist.append(np.linalg.norm(r)/np.linalg.norm(b))
        if hist[-1] < tol: break
    return x, np.array(hist)

def gauss_seidel(A, b, x0=None, tol=1e-10, itmax=10_000):
    A = np.asarray(A, float); b = np.asarray(b, float)
    n = b.size; x = np.zeros(n) if x0 is None else x0.copy()
    hist = []
    for k in range(itmax):
        for i in range(n):
            x[i] = (b[i] - (A[i,:i]@x[:i] + A[i,i+1:]@x[i+1:]))/A[i,i]
        r = b - A @ x
        hist.append(np.linalg.norm(r)/np.linalg.norm(b))
        if hist[-1] < tol: break
    return x, np.array(hist)

# 四、共轭梯度法
## 共轭梯度：从最优化出发

考虑**对称正定（SPD）**的线性方程组

$$
Ax=b,\qquad A=A^\top\succ0.
$$

等价地，最小化严格凸二次型
$$
\min_{x\in\mathbb{R}^n}\ \phi(x)=\tfrac12 x^\top A x - b^\top x.
$$

其梯度 $\nabla\phi(x)=Ax-b$，最优解满足 $Ax^*=b$。

CG 只保证在 **SPD** 情况下正确、单调收敛。若 $A$ 非对称或不定，用 BiCG、GMRES、LSQR 等方法替代。


## 最速下降

**最速下降（Steepest Descent, SD）**每步沿负梯度方向做**一维最小化**
$$
x_{k+1}=x_k+\alpha_k p_k
$$
其中
$$
p_k:=-r_k
$$
其中
$$
r_k:=b-Ax_k
$$
而
$$
\alpha_k=\arg\min_{\alpha}\ \phi(x_k+\alpha p_k)
       = \frac{r_k^\top r_k}{p_k^\top A p_k}.
$$

SD 容易跑出“**锯齿**”且降速慢。

## $A$-共轭及其性质

CG 的关键是构造一组**两两 $A$-共轭（正交）**的方向
$$
p_i^\top A p_j=0\quad(i\ne j),
$$

并在这些方向上做一维最小化。有性质：

- 若 $\{p_0,\dots,p_{n-1}\}$ 线性无关且两两 $A$-共轭，则在精确算术下 **最多 $n$ 步终止**。  
- 第 $k$ 步的解 $x_k$ 是仿射空间 $x_0+\mathcal{K}_k(A,r_0)$ 中 $\phi$ 的最小点，其中
$$
\mathcal{K}_k(A,r_0)=\mathrm{span}\{r_0,Ar_0,\dots, A^{k-1}r_0\}
$$

即 Krylov 子空间。

## 递推推导 CG

我们希望方向 $p_k$ 既像 SD（取 $p_0=-r_0$），又保持 $A$-共轭：
$$
p_{k+1} = -r_{k+1} + \beta_k p_k.
$$

令 $\alpha_k$ 仍为沿 $p_k$ 的线搜索最优步长：
$$
\alpha_k=\frac{r_k^\top r_k}{p_k^\top A p_k}.
$$

更新
$$
x_{k+1}=x_k+\alpha_k p_k,\quad r_{k+1}=r_k-\alpha_k A p_k.
$$

为使 $p_{k+1}$ 与 $p_k$ $A$-共轭（正交），即 $p_{k+1}^\top A p_k=0$，代入得
$$
(-r_{k+1}+\beta_k p_k)^\top A p_k=0
\ \Rightarrow\ 
\beta_k=\frac{r_{k+1}^\top r_{k+1}}{r_k^\top r_k}.
$$

这里使用了 $r_{k+1}^\top A p_k = 0$，可由一维最小化一阶条件推出 $r_{k+1}\perp p_k$。

**由此得到标准 CG 公式：**
$$
\boxed{
\begin{aligned}
&\text{初始化：}\ x_0\ \text{给定},\ r_0=b-Ax_0,\ p_0=r_0.\\
&\text{for } k=0,1,2,\dots\\
&\qquad \alpha_k=\frac{r_k^\top r_k}{p_k^\top A p_k},\quad
x_{k+1}=x_k+\alpha_k p_k,\quad
r_{k+1}=r_k-\alpha_k A p_k,\\
&\qquad \beta_k=\frac{r_{k+1}^\top r_{k+1}}{r_k^\top r_k},\quad
p_{k+1}=r_{k+1}+\beta_k p_k.
\end{aligned}}
$$

- 只需 **一次 `matvec`（$A p_k$**）与向量内积 `axpy`，每步代价 $O(\mathrm{nnz}(A))$。
- 在精确算术下，$\{p_k\}$ $A$-共轭、$\{r_k\}$ 两两正交。


## 收敛率与条件数
在 $A$-范数 $\|e\|_A=\sqrt{e^\top A e}$ 下有经典界（Chebyshev 多项式论证）：
$$
\boxed{\quad
\frac{\|x_k-x^*\|_A}{\|x_0-x^*\|_A}
\ \le\ 2\left(\frac{\sqrt{\kappa}-1}{\sqrt{\kappa}+1}\right)^k,\quad
\kappa:=\kappa_2(A).
\quad}
$$

谱越**团簇**、$\kappa$ 越小，收敛越快。

::: callout-note
当 $A$ **对称正定**时，定义

$$
\quad \|x\|_A := \sqrt{x^\top A x}
$$

* 这是一个合法的范数，正定性来自 $A\succ 0$）。
* 在最小化二次型 $\phi(x)=\tfrac12 x^\top A x - b^\top x$ 的场景，$A$-范数有**能量意义**：

  $$
  \|x-x^*\|_A^2 \;=\; 2\big(\phi(x)-\phi(x^*)\big),
  $$

  因为 $Ax^*=b$。所以 CG 收敛等价于目标函数值的单调下降。
:::

## 预条件共轭梯度

选 SPD 预条件子 $M\approx A$，解等价的**左预条件系统**
$$
M^{-1}Ax=M^{-1}b.
$$

把**残差**经 $M^{-1}$ 预处理：
$$
z_k:=M^{-1} r_k\quad(\text{或解 } M z_k=r_k).
$$

预条件共轭梯度（PCG）的修改仅在内积权与 $\beta$ 的定义：
$$
\boxed{
\begin{aligned}
&\text{初始化：}\ r_0=b-Ax_0,\ z_0=M^{-1}r_0,\ p_0=z_0.\\
&\text{for } k=0,1,2,\dots\\
&\qquad \alpha_k=\frac{r_k^\top z_k}{p_k^\top A p_k},\quad
x_{k+1}=x_k+\alpha_k p_k,\quad
r_{k+1}=r_k-\alpha_k A p_k,\\
&\qquad z_{k+1}=M^{-1} r_{k+1},\quad
\beta_k=\frac{r_{k+1}^\top z_{k+1}}{r_k^\top z_k},\quad
p_{k+1}=z_{k+1}+\beta_k p_k.
\end{aligned}}
$$

- 预条件把谱**压缩**、降低有效条件数，从而**显著加速**。  
- 常见 $M$：对角（Jacobi）、IC(0)/ILU(k)、SSOR、几何/代数多重网格（作为外层预条件）等。  
- **注意**：PCG 要求 $M$ 是 **SPD** 且 $M^{-1}$ 的应用要尽可能便宜。


### 注意事项与实践建议
- **SPD**：CG/PCG 依赖于 $A$-内积与共轭性质，非 SPD 会失效或不稳定。  
- **舍入误差**会破坏正交性：可偶尔重置操作，即 $p_k:=r_k$（重启）或采用基于 Lanczos 的实现。  
- **停止准则**：常用 $\|r_k\|/\|b\| \le \varepsilon$。病态问题上可监控 $\|e_k\|_A$ 的界或用真实误差可估计量。  
- **预条件质量和成本的折中**：$M$ 太“好”可能预解太贵，总体迭代时间最小才是目标。  
- **缩放/对称化**：简单的列/行缩放可改善数值性，右/左预条件需保持等价 SPD 结构。

## SD/CG 在二维二次型上的步进“轨迹”图

In [ ]:
# 2D 二次型
import numpy as np
import matplotlib.pyplot as plt

A = np.array([[5.0, 2.0], [2.0, 1.5]])  # SPD
b = np.array([1.0, 0.5])


def phi(x):
    return 0.5 * x @ A @ x - b @ x


def grad(x):
    return A @ x - b


x_sd = np.array([1.5, -1.0])  # 初始点
x_cg = x_sd.copy()

# 采样等高线
xx = np.linspace(-1.5, 1.8, 200)
yy = np.linspace(-1.5, 1.5, 200)
X, Y = np.meshgrid(xx, yy)
Z = 0.5 * (A[0, 0] * X**2 + 2 * A[0, 1] * X * Y + A[1, 1] * Y**2) - (
    b[0] * X + b[1] * Y
)

# 轨迹
SD, CG = [x_sd.copy()], [x_cg.copy()]
# SD 5 步
for _ in range(5):
    g = grad(x_sd)
    p = -g
    alpha = (g @ g) / (p @ (A @ p))
    x_sd = x_sd + alpha * p
    SD.append(x_sd.copy())
# CG 2 步即可到达解（在 2D 精确算术）
r = b - A @ x_cg
p = r
for _ in range(2):
    Ap = A @ p
    alpha = (r @ r) / (p @ Ap)
    x_cg = x_cg + alpha * p
    r_new = r - alpha * Ap
    beta = (r_new @ r_new) / (r @ r)
    p = r_new + beta * p
    r = r_new
    CG.append(x_cg.copy())

plt.figure(figsize=(6, 4.8))
cs = plt.contour(X, Y, Z, levels=20)
plt.clabel(cs, inline=1, fontsize=8, fmt="%.1f")
SD = np.array(SD)
CG = np.array(CG)
plt.plot(SD[:, 0], SD[:, 1], "-o", label="Steepest Descent")
plt.plot(CG[:, 0], CG[:, 1], "-o", label="Conjugate Gradient")
plt.plot([b[0]], [b[1]], "x", ms=8, label="(not the minimizer)")  # just distractor
x_star = np.linalg.solve(A, b)
plt.plot([x_star[0]], [x_star[1]], "r*", ms=12, label="x*")
plt.legend()
plt.title("SD vs CG on a Quadratic")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

> 观察：SD “锯齿”缓慢逼近；CG 通过**互相 $A$-共轭**的方向快速“直冲”最优点。


## 收敛曲线：PCG 加速示意（1D Poisson）

In [ ]:
def cg(A, b, x0=None, tol=1e-10, itmax=None, M=None):
    A = np.asarray(A, float)
    b = np.asarray(b, float)
    n = b.size
    x = np.zeros(n) if x0 is None else x0.copy()
    r = b - A @ x
    if M is None:
        z = r.copy()
        applyM = lambda v: v
    else:
        D = np.diag(M) if M.ndim == 2 else M
        applyM = lambda v: v / D
        z = applyM(r)
    p = z.copy()
    rz_old = np.dot(r, z)
    hist = [np.linalg.norm(r) / np.linalg.norm(b)]
    if itmax is None:
        itmax = 5 * n
    for k in range(itmax):
        Ap = A @ p
        alpha = rz_old / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        res = np.linalg.norm(r) / np.linalg.norm(b)
        hist.append(res)
        if res < tol:
            break
        z = applyM(r)
        rz_new = np.dot(r, z)
        beta = rz_new / rz_old
        p = z + beta * p
        rz_old = rz_new
    return x, np.array(hist)


def poisson1d(n):
    # A ~ tridiag(-1, 2, -1) ∈ R^{n×n}
    e = np.ones(n)
    A = np.diag(2 * e) - np.diag(e[:-1], 1) - np.diag(e[:-1], -1)
    return A


n = 200
A = poisson1d(n)
x_true = np.ones(n)
b = A @ x_true

xj, hj = jacobi(A, b, tol=1e-10)
xg, hg = gauss_seidel(A, b, tol=1e-10)
xc, hc = cg(A, b, tol=1e-10)
xc2, hc2 = cg(A, b, tol=1e-10, M=np.diag(A))  # Jacobi 预条件

plt.figure(figsize=(6.2, 3.6))
plt.semilogy(hj, label="Jacobi")
plt.semilogy(hg, label="Gauss–Seidel")
plt.semilogy(hc, label="CG")
plt.semilogy(hc2, label="PCG (Jacobi)")
plt.xlabel("iteration")
plt.ylabel("relative residual")
plt.xlim(0, 120)
plt.grid(True, which="both", alpha=0.3)
plt.legend()
plt.title("迭代法收敛曲线（Poisson 1D）")
plt.tight_layout()
plt.show()

> PCG 明显减少迭代步数，预条件压缩了谱（降低有效 $\kappa$）。


## 小结

* CG = “Krylov 子空间最小化 $A$-范误差 + $A$-共轭方向 + 一维最小化”。
* 只适用 **SPD**，非 SPD 需更换算法。
* 收敛快慢 ≈ 谱的团簇程度与条件数，**预条件**是加速核心。
* 实现要点：只需 `A@v` 与一次“预解” `solve(M,·)`，用向量内积与 `axpy` 组装，内存/带宽友好。


# 课后作业

## 回顾

* **直接法**（中小规模/多 RHS）：一次 LU/Cholesky 分解，重复回代，很多数值库默认用部分主元法。
* **条件数**决定从残差到解误差的放大倍率，因此**残差小 ≠ 解误差小**。
* **迭代法**：Jacobi/GS 的 $\rho(T)<1$；CG 适用于 **对称正定阵**，配合预条件可显著加速。
* **批量 RHS**：向量化/并行可大幅提升吞吐。

### a6-1（必做）：编程
实现 **Cholesky** 分解（$A=LL^\top$），并与 `np.linalg.cholesky` 做比较。测试在 SPD 与非 SPD 的情况下的程序行为。


## 附录 A：批量右端与并行

* 直接法：**分解一次**（LU/Cholesky），对 $B=[b_1,\dots,b_m]$ 做两次三角解：$LY=PB$、$UX=Y$。
* 迭代法：同一 $A$ 对不同 $b$ 可**共享矩阵–向量乘**。在矢量化/并行环境下，对多个 RHS 可**矩阵–矩阵乘**。

In [ ]:
# 复用上面的 lu_factor_pp / lu_solve
rng = np.random.default_rng(42)
A = poisson1d(400) + 1e-2 * np.eye(400)
B = rng.standard_normal((400, 20))

L, U, piv = lu_factor_pp(A)
X_my = lu_solve(L, U, piv, B)
X_np = np.linalg.solve(A, B)
print("multi-RHS 误差:", np.linalg.norm(X_my - X_np) / np.linalg.norm(X_np))

## 附录 B：JAX 版 CG、`vmap` 批量 RHS 与 `lax.scan` 时间推进


In [ ]:
try:
    import jax, jax.numpy as jnp
    jax.config.update("jax_enable_x64", True)

    def cg_jax(A, b, x0=None, tol=1e-10, itmax=None, M_diag=None):
        A = jnp.asarray(A); b = jnp.asarray(b)
        n = b.size; 
        x = jnp.zeros(n) if x0 is None else jnp.asarray(x0)
        if itmax is None: itmax = 5*n

        def applyM(v):
            return v if M_diag is None else v / M_diag

        r0 = b - A @ x
        z0 = applyM(r0); p0 = z0
        rz0 = jnp.vdot(r0, z0).real
        bnorm = jnp.linalg.norm(b)

        def body(carry, _):
            x, r, p, rz_old = carry
            Ap = A @ p
            alpha = rz_old / jnp.vdot(p, Ap).real
            x_new = x + alpha * p
            r_new = r - alpha * Ap
            z_new = applyM(r_new)
            rz_new = jnp.vdot(r_new, z_new).real
            beta = rz_new / rz_old
            p_new = z_new + beta * p
            return (x_new, r_new, p_new, rz_new), jnp.linalg.norm(r_new)/bnorm

        (x_fin, r_fin, _, _), hist = jax.lax.scan(body, (x, r0, p0, rz0), xs=None, length=itmax)
        return x_fin, jnp.array(hist)

    # vmap 批量 RHS
    n = 200
    A = jnp.array(poisson1d(n))
    rng = np.random.default_rng(0)
    B = jnp.array(rng.standard_normal((n, 8)))
    Mdiag = jnp.diag(A)

    solve_one = lambda b: cg_jax(A, b, tol=1e-10, itmax=500, M_diag=Mdiag)[0]
    solve_many = jax.vmap(solve_one, in_axes=1, out_axes=1)  # 按列映射
    X = solve_many(B)

    print("JAX batch shapes:", B.shape, X.shape)

except Exception as e:
    print("JAX not available or skipped:", e)

**说明**

* `lax.scan` 以时间步推进迭代，`vmap` 对列批量并行。
* 在 CPU 环境上，JIT 后的稠密 `matvec`/批量计算通常比 Python 循环更快，对稀疏矩阵则需专门库。